In [1]:
import requests
import pandas as pd
from sqlalchemy import create_engine

### Extract data from World Bank API
Get external data owed by Kenya from 2010 to 2024 using the DT.DOD.DECT.CD indicator as a parameter.


In [8]:
url = "https://api.worldbank.org/v2/country/KE/indicator/DT.DOD.DECT.CD"
params = {
    'date': '2010:2024',
    'format': 'json'
}

response = requests.get(url, params=params)
if response.status_code == 200:
    rel_data = []
    data = response.json()
    for item in data[1]:
        rel_data.append({
            'countryName': item['country']['id'],
            'year': item['date'],
            'debtValue': item['value'],
            'indicator': item['indicator']['value']
        })
    print(rel_data)
else:
    print(f"Error fetching data: {response.status_code}, {response.text}")

[{'countryName': 'KE', 'year': '2024', 'debtValue': None, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2023', 'debtValue': 42910026054.2, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2022', 'debtValue': 41556299639.8, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2021', 'debtValue': 41219100011.5, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2020', 'debtValue': 38038451074.1, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2019', 'debtValue': 34970099589.9, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2018', 'debtValue': 31433802015.7, 'indicator': 'External debt stocks, total (DOD, current US$)'}, {'countryName': 'KE', 'year': '2017', 'debtValue': 26951349499.4, 'indicator': 'External debt stoc

In [9]:
df = pd.DataFrame(rel_data)
df.head()

,countryName,year,debtValue,indicator
0,KE,2024,NaN,"External debt stocks, total (DOD, current US$)"
1,KE,2023,4.291003e+10,"External debt stocks, total (DOD, current US$)"
2,KE,2022,4.155630e+10,"External debt stocks, total (DOD, current US$)"
3,KE,2021,4.121910e+10,"External debt stocks, total (DOD, current US$)"
4,KE,2020,3.803845e+10,"External debt stocks, total (DOD, current US$)"


In [10]:
# drop first row with NaN value
df.dropna(inplace=True)
df.head()

,countryName,year,debtValue,indicator
1,KE,2023,4.291003e+10,"External debt stocks, total (DOD, current US$)"
2,KE,2022,4.155630e+10,"External debt stocks, total (DOD, current US$)"
3,KE,2021,4.121910e+10,"External debt stocks, total (DOD, current US$)"
4,KE,2020,3.803845e+10,"External debt stocks, total (DOD, current US$)"
5,KE,2019,3.497010e+10,"External debt stocks, total (DOD, current US$)"


In [12]:
df.dtypes

countryName     object
year            object
debtValue      float64
indicator       object
dtype: object

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
DB_URL = os.getenv("DB_URL")

engine = create_engine(DB_URL)
try:
    df.to_sql('external_debt', con=engine, index=False, if_exists='replace')
    print("Data loaded successfully!")
except Exception as e:
    print(f"Loading into Postgres DB error: {e}")